# The evil Professor

> This exercise is courtesy of Prof. Lombardi

Prof. X has published in a single shot the dates of the exam for his course for whole year. While officially presented as a student-friendly measure (so they can better plan their activities), it is in fact all part of an *evil* plan to achieve the best results with minimum effort.

In fact, Prof. X finals contain two exercises (A and B), which can be **safely recycled** between two dates. 

HOWEVER... this assumes that **no student attends both dates**, which is of course what John, Jerry and Jake have promptly done.

By carefully examining the student lists of the 5 available exams, X notices that:

- John is in the list for date 1, 3, and 5
- Jerry is in the list for date 1 and 2
- Jake is in the list for date 2 and 4

Additionally, lest the dreaded rumor that Prof. X’s exams all look the same starts to spread, **no exercise can appear in the same position twice** (e.g. exercise 1 cannot appear twice in position A).

Will X manage to prepare all finals **using only 6 exercises**? Model the problem using Constraint Programming and show a possible solution.

# Preparation...

First of all, get sure you have installed OR-Tools, a quite of Optimizaiotn research tools by Google:
<https://developers.google.com/optimization>

Simply type the command
`pip install ortools`

Just in case, see <https://developers.google.com/optimization/install> for detailed instructions.


---

# Solution

## Getting sense of the variables...

Which are the variables? and which are the values? Let us focus on our goal:

> Will X manage to prepare all finals using only 6 exercises?  
> There are 5 dates...  
> ... each date has an exercise A and an exercise B  

In other words, for each exercise A and B of each date, we have to asign on among the avialable exercises 1..6.

## Variables

For each date (there are five of them), will have a variable Ai and a variable Bi. In total, ten variables:

`A1, B1, A2, B2, A3, B3, A4, B4, A5, B5`

## Domains

Each variable should be assigned one among the six available exercises... domains: `[1..6]`

## Creating the model in OR-Tool

The google library provides two main groups of methods:

- methods for modeling the problem
- methods for solviong the problem

Let us start with modeling the problem. First of all, it is necessary to create an (initally empty) model:

In [1]:
from ortools.sat.python import cp_model

# the model variable will contain our model
model = cp_model.CpModel()

Then we need to add the variables to our model. Several different types of variables can be added:
- boolean variables: `NewBoolVar(self, name)`
- integer variables: `NewIntVar(self, lb, ub, name)`
- integer variables over domains with possible holes: `NewIntVarFromDomain(self, domain, name)`
- intervals: `NewIntervalVar(self, start, size, end, name)`
- optional intervals: `NewOptionalIntervalVar(self, start, size, end, is_present, name)`

We have ten variables, whose domains are all 1..6 ... we shall go for the NewIntVar method. 

In [2]:
# Adding the variables to the model

var_names = ['a1', 'b1', 'a2', 'b2', 'a3', 'b3', 'a4', 'b4', 'a5', 'b5']
# can you imagine a faster way for generating the variable names?

a1 = model.NewIntVar(1, 6, 'a1')
b1 = model.NewIntVar(1, 6, 'b1')
a2 = model.NewIntVar(1, 6, 'a2')
b2 = model.NewIntVar(1, 6, 'b2')
a3 = model.NewIntVar(1, 6, 'a3')
b3 = model.NewIntVar(1, 6, 'b3')
a4 = model.NewIntVar(1, 6, 'a4')
b4 = model.NewIntVar(1, 6, 'b4')
a5 = model.NewIntVar(1, 6, 'a5')
b5 = model.NewIntVar(1, 6, 'b5')


# Alternatively, you could write:
# for v in var_names:
#    model.NewIntVar(1, 6, v)
# but then, is tougher to post the constraints later on


## Making sense of the constraints (1/2)

Prof. X wants to avoid that a student attending two dates will receive the same exam. We know that:
> - John is in the list for date 1, 3, and 5
> - Jerry is in the list for date 1 and 2
> - Jake is in the list for date 2 and 4

Due to John's strategy, we derive that exercises for dates 1,3, and 5, must be all different. This can be achieved by assing an all_different constraint:
`AddAllDifferent(self, variables)`, where `variables` is a list of integer variables.  
We will do the same for dates 1 and 2 (due to Jerry's strategy), and for dates 2 and 4 (due to Jake's strategy).

In [3]:
model.AddAllDifferent([a1,b1,a3,b3,a5,b5])
model.AddAllDifferent([a1,b1,a2,b2])
model.AddAllDifferent([a2,b2,a4,b4])

## Making sense of the constraints (2/2)

We are still missing a (group of) constraints... from the original text:
> no exercise can appear in the same position twice

We can add this constraint by simply specifying that A1..A6 should be all different...


In [4]:
model.AddAllDifferent([a1,a2,a3,a4,a5])
model.AddAllDifferent([b1,b2,b3,b4,b5])

## Computing a solution

At this point, we can ask the solver to compute a solution for us. This is done through two steps:
1. First, we instantiate a solver: `solver = cp_model.CpSolver()`
2. We ask the solver to solve our model: `status = solver.Solve(model)`

Then, we will observe the status: it will tell us if a solution was found or not. Possible values of status:
- `cp_model.OPTIMAL`
- `cp_model.FEASIBLE`
- `cp_model.INFEASIBLE`
- `cp_model.MODEL_INVALID`
- `cp_model.UNKNOWN`

If the status is `OPTIMAL` or `FEASIBLE`, we can ask for solutions and print them.

In [5]:
solver = cp_model.CpSolver()

status = solver.Solve(model)

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print('a1 = %i' % solver.Value(a1))
    print('b1 = %i' % solver.Value(b1))
    print('a2 = %i' % solver.Value(a2))
    print('b2 = %i' % solver.Value(b2))
    print('a3 = %i' % solver.Value(a3))
    print('b3 = %i' % solver.Value(b3))
    print('a4 = %i' % solver.Value(a4))
    print('b4 = %i' % solver.Value(b4))
    print('a5 = %i' % solver.Value(a5))
    print('b5 = %i' % solver.Value(b5))
else:
    print('No solution found.')

a1 = 1
b1 = 2
a2 = 4
b2 = 3
a3 = 6
b3 = 5
a4 = 2
b4 = 1
a5 = 3
b5 = 4


## Printing and counting all solutions

What if we want to print and count all solutions?

Google provides a mechanism thorugh a callback function: for foudn solution, the callback function will be invoked.

An example (provided by Google itself) is the following one:


    class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):
        """Print intermediate solutions."""

        def __init__(self, variables):
            cp_model.CpSolverSolutionCallback.__init__(self)
            self.__variables = variables
            self.__solution_count = 0

        def on_solution_callback(self):
            self.__solution_count += 1
            for v in self.__variables:
                print('%s=%i' % (v, self.Value(v)), end=' ')
            print()

        def solution_count(self):
            return self.__solution_count

    solver = cp_model.CpSolver()
    solution_printer = VarArraySolutionPrinter([x, y, z])

    solver.parameters.enumerate_all_solutions = True

    status = solver.Solve(model, solution_printer)


In [7]:
class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0

    def on_solution_callback(self):
        self.__solution_count += 1
        # for v in self.__variables:
            # print('%s=%i' % (v, self.Value(v)), end=' ')
        # print()

    def solution_count(self):
        return self.__solution_count

solver = cp_model.CpSolver()
solution_printer = VarArraySolutionPrinter([a1, b1, a2, b2, a3, b3, a4, b4, a5, b5])

solver.parameters.enumerate_all_solutions = True
status = solver.Solve(model, solution_printer)
print('Solutions found:', solution_printer.solution_count())

Solutions found: 11520
